In [ ]:
# 1️⃣ Importar librerías
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
import numpy as np
import uuid

# Modelo de embeddings
# model_name = "sentence-transformers/all-MiniLM-L6-v2" Este modelo no esta entrenado para Español
model_name = "paraphrase-multilingual-MiniLM-L12-v2"

model = SentenceTransformer(model_name)

def get_embeddings(text):
    return model.encode(text).tolist()

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos():
    docs_path = "docs/"
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}
    
    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")

def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=1
    )
    
    if results["documents"]:
        print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
        print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
        print(f"🆔 ID: {results['ids'][0][0]}")
    else:
        print("⚠️ No se encontraron resultados.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    metadata = {"source": "documento_actualizado.txt"}
    
    collection.add(
        documents=[new_text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[new_id]
    )
    print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

print('listo')

/Users/eMinus1db/Documents/Python_Labs/sandbox_ai/data_bases_ai/bases-datos-IA/venv-BDIA/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


listo


In [25]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [26]:
get_embeddings("Reynaldo González es un ingeniero de software con 10 años de experiencia en desarrollo web y móvil.")

[-0.13363400101661682,
 0.09824267774820328,
 -0.038939330726861954,
 -0.09957413375377655,
 -0.10088636726140976,
 -0.09350062161684036,
 -0.10539218783378601,
 0.44167560338974,
 0.3837745487689972,
 0.11706008017063141,
 -0.16925598680973053,
 -0.03680584579706192,
 -0.06929896026849747,
 0.16074343025684357,
 0.06067797541618347,
 0.1504400670528412,
 -0.13210487365722656,
 0.007339669857174158,
 0.004763266537338495,
 -0.2871944308280945,
 0.13415353000164032,
 0.05589154362678528,
 -0.05534923076629639,
 -0.14180582761764526,
 -0.208276629447937,
 0.10395640879869461,
 0.015970567241311073,
 -0.019185690209269524,
 0.09457262605428696,
 -0.22653546929359436,
 0.16219882667064667,
 0.21008072793483734,
 0.49164697527885437,
 -0.10322771966457367,
 -0.25187841057777405,
 0.15674416720867157,
 -0.026113061234354973,
 0.022394152358174324,
 -0.12019314616918564,
 0.012447958812117577,
 -0.12207493931055069,
 -0.17883805930614471,
 0.06515374034643173,
 -0.14968083798885345,
 0.033834

In [27]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [28]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: a07dd390-09d9-4235-ab01-cc797272a28b, Contenido: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El a  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.0520102  -0.04184369  0.03102789  0.03516814  0.00390169]...
ID: 727d58d6-e5c9-489e-86f4-11d95b9e3bf8, Contenido: ciosa continuaba, con el tráfico constante y el murmullo de las conversaciones. Los turistas se mara  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06561408  0.00274146  0.01852137 -0.00384712 -0.08065522]...
ID: 93c89342-60f8-44b8-ac20-db0e01be46e7, Contenido:  los restaurantes servían deliciosas parrilladas.

Hacia el norte, el elegante barrio de Recoleta in  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06329247 -0.05760502  0.01019358  0.01427122 -0.11836312]...
ID: b66e4266-d216-4bd7-a9e6-e6dde0e61152, Contenido: y novelas exploran la realidad, la fantasía y la condición humana con una prosa exquisita.

El fútbo  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.10750

In [ ]:
while True:
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
        
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break

### **Ejercicio 1: Crear y Consultar** ###

**Crea un documento nuevo con el texto: "La gravedad es una fuerza fundamental en el universo".**


In [30]:
create_example("La gravedad es una fuerza fundamental en el universo")

✅ Documento añadido con ID: 93640ef2-29d7-436f-9e4c-79f2885e766a


In [31]:
results = collection.get()

print("IDs:", results['ids'])
print("Documents:", results['documents'])

IDs: ['a07dd390-09d9-4235-ab01-cc797272a28b', '727d58d6-e5c9-489e-86f4-11d95b9e3bf8', '93c89342-60f8-44b8-ac20-db0e01be46e7', 'b66e4266-d216-4bd7-a9e6-e6dde0e61152', '922aa459-5ec9-461d-b2ab-68dddfe117b7', 'f88605a1-f5a3-4b00-98ee-d9fa7b7811e4', '56b13237-d211-47bb-8ef2-77599a5d9b0f', '45aed722-c976-4f03-b264-5df7adce03cb', '57364637-f410-4177-8f24-87227a0571aa', 'bf4f278d-b072-4700-b0d5-444ba52adf45', '2016ed1b-e0ab-4ebf-8eb3-8f2f3eed1b3f', '8998b0c3-21a8-450b-ad8e-6c428068b880', 'a6e3b550-9154-4589-9fd6-192ae38411f1', '2510bbfd-e5e0-447e-a2ad-a851ba45ce85', 'b0085745-e5ea-4207-9620-fdb59df2cea6', 'fe8f63b1-b010-441e-ae53-2111d5f0d6c4', 'e9c9fecd-8bde-4413-8fd2-92457bc30364', '51883084-8ec1-4887-ba96-d772dfa4bb1a', 'c8576df9-90fc-4ff5-a920-8112d8dc2468', '1cbd90b2-ff75-4949-907d-fb07a78b5bac', '6ab0708a-30b6-47ff-ad85-15653ddbb1b9', '64802144-8b4e-4fef-bd56-11e66a2dfa2b', '341d8543-cc66-479f-8256-fa18ead0f326', '5da5a94a-7d58-4816-9cbe-2ec1f52c3615', 'c449e973-1f2c-49a6-b377-9c54ac2a2

**Realiza una consulta para encontrar documentos similares a la frase: "Fuerzas naturales que gobiernan el cosmos".**

In [32]:
read_example("Fuerzas naturales que gobiernan el cosmos")

📄 Mejor coincidencia: La gravedad es una fuerza fundamental en el universo
📁 Fuente: nuevo_documento.txt
🆔 ID: 93640ef2-29d7-436f-9e4c-79f2885e766a


**¿Qué tipo de operación CRUD usaste primero?**

La primera operación consistio en crear un nuevo documento, por tanto fue una operacion de Creacion o CREATE.

**¿Cuál es el ID del documento creado? (Usa collection.peek()).**

ID: 93640ef2-29d7-436f-9e4c-79f2885e766a

**Explica por qué el documento aparece como coincidencia en la consulta.**

En realidad, utilizando el modelo planteado en el ejercicio ("sentence-transformers/all-MiniLM-L6-v2") el documento introducido recientemente (La gravedad es una fuerza fundamental en el universo) no aparecia en absoluto como una coincidencia. Aparecian en su lugar documentos que no tenian ningún elemento en común con el documento introducido. Realizando una investigación acerca de este problema encontré que el modelo planteado en el ejercicio ("sentence-transformers/all-MiniLM-L6-v2") no está entrenado para el idioma Español, por tanto tiene sentido que las coincidencias sean bastante pobres.

En lugar del ese modelo, utilicé este otro: "paraphrase-multilingual-MiniLM-L12-v2" el cual si se encuentra entrenado para el idioma Español. Luego de volver a crear los embeddings y volver a crear la base de datos, volví a introducir el documento y a realizar al consulta y en este caso devolvió la respuesta esperada, es decir que encontro que la frase de la consulta "Fuerzas naturales que gobiernan el cosmos" coincide con el documento "La gravedad es una fuerza fundamental en el universo" por que habla del concepto de Fuerza Natural dentro del contexto del cosmos o universo.

### **Ejercicio 2: Actualizar y Eliminar** ###

**Crea un documento con el texto: "Redes neuronales imitan procesos cerebrales".**

In [57]:
create_example("Redes neuronales imitan procesos cerebrales")

✅ Documento añadido con ID: 33a42fc4-b511-4847-be14-05eb8f1cfb46


**Encuentra su ID, luego actualízalo por: "Deep Learning es un subcampo de IA con redes neuronales profundas".**

In [58]:
sample = collection.get()

sample_doc = "Redes neuronales imitan procesos cerebrales"

for i, doc in enumerate(sample['documents']):
    if doc == sample_doc:
        print(f"Documento encontrado con ID: {sample['ids'][i]}")
        break
else:
    print("Documento no encontrado")

Documento encontrado con ID: 33a42fc4-b511-4847-be14-05eb8f1cfb46


In [59]:
update_example("33a42fc4-b511-4847-be14-05eb8f1cfb46", "Deep Learning es un subcampo de IA con redes neuronales profundas")

♻️ Documento actualizado. Nuevo ID: b9c227a7-338e-4e43-be10-1f20cfd9c39b


In [60]:
sample = collection.get()

sample_id = "33a42fc4-b511-4847-be14-05eb8f1cfb46"

for i, doc in enumerate(sample['ids']):
    if doc == sample_id:
        print(f"Documento encontrado: {sample['documents'][i]}")
        break
else:
    print("Documento no encontrado")

Documento no encontrado


**Elimina el documento original usando el mismo ID.**

In [61]:
delete_example("33a42fc4-b511-4847-be14-05eb8f1cfb46")

🗑️ Documento con ID '33a42fc4-b511-4847-be14-05eb8f1cfb46' eliminado.


In [62]:
sample = collection.get()

sample_id = "33a42fc4-b511-4847-be14-05eb8f1cfb46"

for i, doc in enumerate(sample['ids']):
    if doc == sample_id:
        print(f"Documento encontrado: {sample['documents'][i]}")
        break
else:
    print("Documento no encontrado")

Documento no encontrado


**¿Qué ocurre si intentas actualizar un documento eliminado?**

In [63]:
update_example("33a42fc4-b511-4847-be14-05eb8f1cfb46", "Intentando actualizar un documento eliminado")

♻️ Documento actualizado. Nuevo ID: 046162d8-0f70-4503-a28b-8f9ab6f28de5


In [64]:
sample = collection.get()

sample_id = "046162d8-0f70-4503-a28b-8f9ab6f28de5"

for i, doc in enumerate(sample['ids']):
    if doc == sample_id:
        print(f"Documento encontrado: {sample['documents'][i]}")
        break
else:
    print("Documento no encontrado")

Documento encontrado: Intentando actualizar un documento eliminado


La actualización se puede completar de forma exitosa. Esto sucede porque cuando un documento es eliminado, Chroma marca el docuemnto como internamente borrado, sin embargo la data del docuemnto no es eliminada de forma inmediata. El documento en cuestion puede de hecho continuar en la base de datos, simplemente no se encuentra indexado para realizar búsquedas.

**¿Cómo se garantiza la integridad de los datos en las operaciones?**

Chroma mantiene una capa de indexación separada para las operaciones de búsquedas y consultas. Esta capa se asegura que solo documentos que no han sido eliminados son utilizados en búsquedas.

### **Ejercicio 3: Errores y Validación** ###

**Intenta eliminar un documento con el ID "no_existe".**

In [65]:
delete_example("no_existe")

🗑️ Documento con ID 'no_existe' eliminado.


**Crea un nuevo documento sin usar get_embeddings() (ejemplo: enviar un array vacío).**

In [66]:
create_example()

TypeError: create_example() missing 1 required positional argument: 'new_doc'

**¿Qué errores ocurren en cada caso?**

En el caso de eliminar un documento con un ID inexistente, Chroma permite realizar el borrado sin retornar ningún tipo de mensaje de error.

Para el caso de crear un documento sin el argumento correspondiente al nombre, en efecto si se retorna un mensaje de error, sin embargo este corresponde mas a la sintaxis de Python que espera un argumento posicional para la funcion "create_example()".

**¿Cómo prevenir estos problemas?**

En ambos casos se pueden prevenir estos problemas introduciendo mecanismos de validación de las variables de entrada. Adicionalmente a esto, se puede poner cada función que interactua con la base de datos dentro de una sección Try and Except para poder manejar las excepciones donde una variable no válida sea introducida.

### **Ejercicio 4: Consultas por Tema** ###

**Crea tres documentos:**
- "Python es ideal para ciencia de datos"
- "JavaScript maneja interacciones web dinámicas"
- "Java se usa en aplicaciones empresariales"

In [67]:
create_example("Python es ideal para ciencia de datos")

✅ Documento añadido con ID: f2b02210-a5f0-4f59-bca1-280b12173d31


In [68]:
create_example("JavaScript maneja interacciones web dinámicas")

✅ Documento añadido con ID: f76a6214-1a0d-4871-91f2-fe81e65aaf9b


In [69]:
create_example("Java se usa en aplicaciones empresariales")

✅ Documento añadido con ID: 18d57d67-cdf9-46a2-bc9d-998084bf28df


**Realiza una consulta por la frase: "Lenguajes backend y frontend".**

In [70]:
read_example("Lenguajes backend y frontend")

📄 Mejor coincidencia: ecialmente útil para buscar información relevante en grandes colecciones de datos.

La consulta de una base de datos vectorial implica buscar los embeddings más cercanos al embedding de la consulta. La distancia entre los embeddings indica la similitud semántica entre los datos.

Las aplicaciones de las bases de datos vectoriales son diversas, incluyendo la búsqueda semántica, los sistemas de recomendación, la detección de anomalías y el procesamiento del lenguaje natural.

El procesamiento del 
📁 Fuente: doc1.txt
🆔 ID: 45aed722-c976-4f03-b264-5df7adce03cb


**¿Cuáles documentos aparecen como coincidencias?**

Los documentos que aparecen como coincidencias corresponden a los documentos con los que inicialmente se creo la base de datos. Es decir que ninguno de los documentos recientemente creados en este ejercicio son encontrados como coincidencias.

**Analiza cómo las palabras clave influyen en los resultados.**

Aparentemente, la palabra "Lenguaje" no resulta ser asociada a los nombres de lenguajes como "Python" o "JavaScript" por ejemplo. Se observa que los resultados guardan relacion más con la palabra "backend", esto porque lo detallado en cada uno de los documentos de la respuesta coincide con las descripciones de las funciones del backend.

Resulta curioso que el modelo no vincule "Lenguajes" con "Python", "Java" o "JavaScript", quizás sean necesarias mas referencias que vinculen los lenguajes de programación con la palabra "Lenguaje".

### **Ejercicio 5: Eliminaciones Selectivas** ###

**Crea dos documentos con tópicos opuestos (ejemplo: uno sobre "Ecología" y otro sobre "Minería de datos").**

**Elimina el documento sobre ecología usando su ID.**

**Verifica que solo quede un documento.**

**¿Cómo usar collection.count() para confirmar? 2**


### **Ejercicio 6: Actualización en Cadena** ###

**Crea un documento con el texto "IA y ética son temas actuales".**

**Actualízalo dos veces:**

**Primera actualización: "Ética en IA aborda dilemas morales"**

**Segunda actualización: "El uso de IA debe estar regulado por principios éticos".**

**¿Cómo sigue evolucionando el contenido del documento con cada update?**

### **Ejercicio 7: Consultas Ambiguas** ###

**Crea dos documentos similares pero no idénticos:**

**"Blockchain garantiza transacciones seguras"**

**"Criptomonedas usan tecnología blockchain".**

**Realiza una consulta por "Seguridad en finanzas digitales".**

**¿Ambos documentos aparecen? Explica la similitud semántica.**

### **Ejercicio 8: Reiniciar Base de Datos** ###

**Elimina todos los documentos usando delete() con el parámetro correcto.**

**Vuelve a cargar la base desde cero con crear_base_datos().**

**¿Cuántos documentos hay ahora? (Usa collection.count()).**


### **Ejercicio 9: Errores de Inserción** ###

**Intenta crear un documento sin texto (ejemplo: cadena vacía).**

**Corrige el error usando la función get_embeddings().**

**¿Qué lección aprendemos sobre los requisitos de ChromaDB?**